In [1]:
import carla
import time
from intersection_scenario_manager import IntersectionScenario
import numpy as np
import sys
sys.path.append("cartesian_to_frenet/")
from topology_extraction import get_opendrive_tree, get_junction_topology, get_junction_roads_topology
sys.path.append("../../global_route_planner/")
from global_planner import get_client, draw_waypoints, spawn_vehicle

In [2]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)

In [14]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

all_waypoints = world.get_map().generate_waypoints(1)
def draw(road_id, lane_id=None, color=[0,255,0]):
    for wp in all_waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            draw_waypoints(world, [wp], 10, color) 
#             print(wp.lane_id)
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            draw_waypoints(world, [wp], 10, color) 
            
tree = get_opendrive_tree(world)
junction_topology = get_junction_topology(tree)
road_topology = get_junction_roads_topology(tree)

Connection to CARLA server established!


In [4]:
SM = IntersectionScenario(client)

Intersection Manager Initialized...


dict_keys([224, 965, 421, 1175, 905, 1162, 139, 1260, 685, 334, 751, 1148, 1050, 53, 599, 1070, 943, 509, 924, 829, 245])

In [5]:
junctionId = 53

# for idx in range(len(junction_topology[junctionId])):
#     intersection_road_id, intersection_lane_id = junction_topology[junctionId][idx][1]

#     road_1_id, road_2_id, lane_connections = road_topology[intersection_road_id]
#     print(road_1_id, road_2_id, intersection_road_id, lane_connections)

In [27]:
a,b,c,d,e,f = SM.reset(num_vehicles=10, warm_start_duration=20, junction_id=junctionId)

generating scenario at junction id:  53
Response Error while applying batch!
Control handed to system....


In [53]:
draw_waypoints(world, f[:5]+f[51:], life_time=10)

In [36]:
def waypoint_to_pose2D(waypoint):

    x = waypoint.transform.location.x
    y = waypoint.transform.location.y
    theta = waypoint.transform.rotation.yaw * np.pi / 180

    return Pose2D(x=x, y=y, theta=theta)

In [20]:
f

In [21]:
global_path = [
    waypoint_to_pose2D(wp) for wp in f
]
global_path = [GlobalPathPoint(global_pose=pose) for pose in global_path]
global_path = GlobalPath(path_points=global_path)

NameError: name 'Pose2D' is not defined

In [19]:
draw(41,1)
draw(22,-1)
draw(151,-1)

In [15]:
intersecting_left, intersecting_right, parallel_same_dir, parallel_opposite_dir = d
# parallel_same_dir.append(e)

In [33]:
for key in c:
    draw(key[0], key[1])

In [35]:
ego_nearest_waypoint = client.get_world().get_map().get_waypoint(
    a.get_location(), project_to_road=True
)
client.get_world().debug.draw_string(
    ego_nearest_waypoint.transform.location,
    "O",
    draw_shadow=False,
    color=carla.Color(r=255, g=0, b=0),
    life_time=100,
)

In [22]:
for elem in parallel_opposite_dir:
    if(len(elem) == 2):
        draw(elem[0], elem[1])
    else:
        draw(elem[0][0],elem[0][1])
        draw(elem[1][0],elem[1][1])
        draw(elem[2][0],elem[2][1])

In [23]:
a.get_traffic_light_state()

carla.libcarla.TrafficLightState.Green

In [13]:
# full_parallel_opposite_dir = []

# for key in parallel_opposite_dir:
    
#     if(key not in c):
#         continue
        
#     for elem in c[key]:
#         if (elem[0], elem[1]) in parallel_opposite_dir:
#             full_parallel_opposite_dir.append([key, (elem[2], elem[3][0]), (elem[0], elem[1])])
            
# if len(full_parallel_opposite_dir) == 0:
#     full_parallel_opposite_dir = parallel_opposite_dir

            
# full_parallel_same_dir = []

# for key in parallel_same_dir:
    
#     if(key not in c):
#         continue
        
#     for elem in c[key]:
#         if (elem[0], elem[1]) in parallel_same_dir:
#             full_parallel_same_dir.append([key, (elem[2], elem[3][0]), (elem[0], elem[1])])

# if len(full_parallel_same_dir) == 0:
#     full_parallel_same_dir = parallel_same_dir
            
        
# full_intersecting_right = []

# for key in intersecting_right:
    
#     if(key not in c):
#         continue
        
#     for elem in c[key]:
#         if (elem[0], elem[1]) in intersecting_right:
#             full_intersecting_right.append([key, (elem[2], elem[3][0]), (elem[0], elem[1])])
    
# if len(full_intersecting_right) == 0:
#     full_intersecting_right = intersecting_right
    
    
# full_intersecting_left = []

# for key in intersecting_left:
    
#     if(key not in c):
#         continue
        
#     for elem in c[key]:
#         if (elem[0], elem[1]) in intersecting_left:
#             full_intersecting_left.append([key, (elem[2], elem[3][0]), (elem[0], elem[1])])
    
# if len(full_intersecting_left) == 0:
#     full_intersecting_left = intersecting_left
    

        
        
    
    

In [100]:
for elem in full_intersecting_left:
    if(len(elem) == 2):
        draw(elem[0], elem[1])
    else:
        draw(elem[0][0],elem[0][1])
        draw(elem[1][0],elem[1][1])
        draw(elem[2][0],elem[2][1])

In [45]:
for key in parallel_same_dir:
    draw(key[0], key[1])

(52, -2)
(52, -1)
(51, -2)


In [25]:
a.destroy()
for actor in b:
    actor.destroy()